## Similarity Exercise

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from sentence_transformers import SentenceTransformer
import faiss

In this exercise, you've been provided the title and abstract of 500 recent machine learning research papers posted on arXiv.org.

In [2]:
articles = pd.read_csv('../data/arxiv_papers.csv')
articles.head()

,title,abstract,url
0,GoT-R1: Unleashing Reasoning Capability of MLL...,Visual generation models have made remarkable ...,http://arxiv.org/abs/2505.17022v1
1,Delving into RL for Image Generation with CoT:...,Recent advancements underscore the significant...,http://arxiv.org/abs/2505.17017v1
2,Interactive Post-Training for Vision-Language-...,"We introduce RIPT-VLA, a simple and scalable r...",http://arxiv.org/abs/2505.17016v1
3,When Are Concepts Erased From Diffusion Models?,"Concept erasure, the ability to selectively pr...",http://arxiv.org/abs/2505.17013v1
4,Understanding Prompt Tuning and In-Context Lea...,Prompting is one of the main ways to adapt a p...,http://arxiv.org/abs/2505.17010v1


In [3]:
i = 0
print(f'Title: {articles.loc[i,"title"]}\n')
print(f'Text: {articles.loc[i,"abstract"]}')

Title: GoT-R1: Unleashing Reasoning Capability of MLLM for Visual Generation with Reinforcement Learning

Text: Visual generation models have made remarkable progress in creating realistic
images from text prompts, yet struggle with complex prompts that specify
multiple objects with precise spatial relationships and attributes. Effective
handling of such prompts requires explicit reasoning about the semantic content
and spatial layout. We present GoT-R1, a framework that applies reinforcement
learning to enhance semantic-spatial reasoning in visual generation. Building
upon the Generation Chain-of-Thought approach, GoT-R1 enables models to
autonomously discover effective reasoning strategies beyond predefined
templates through carefully designed reinforcement learning. To achieve this,
we propose a dual-stage multi-dimensional reward framework that leverages MLLMs
to evaluate both the reasoning process and final output, enabling effective
supervision across the entire generation pipeli

Let's try out a variety of ways of vectorizing and searching for semantically-similar papers.

### Method 1: Bag of Words

Fit a CountVectorizer to the abstracts of the articles with all of the defaults.  Then vectorize the dataset using the fit vectorizer. 

In [37]:
vect = CountVectorizer()
X = vect.fit_transform(articles['abstract'])

In [38]:
type(X)

scipy.sparse._csr.csr_matrix

**Question:** How many dimensions do the embeddings have?

In [39]:
X.shape[1]

7978

Now, let's use the embeddings to look for similar articles to a search query.

Apply the vectorizer you fit earlier to this query string to get an embedding. 

**Hint:** You can't pass a string to a vectorizer, but you can pass a list containing a string.

In [41]:
query = "vector databases for retrieval augmented generation"
query_list = [query]
y = vect.transform(query_list)

In [42]:
query_list

['vector databases for retrieval augmented generation']

Now, we need to find the similarity between our query embedding and each vectorized article.

For this, you can use the [cosine similarity function from scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)

Calculate the similarity between the query embedding and each article embedding and save the result to a variable named `similarity_scores`.

In [43]:
similarity_scores = cosine_similarity(X, y)
similarity_scores

array([[0.11020775],
       [0.08461622],
       [0.01780047],
       [0.04145133],
       [0.03806935],
       [0.0814463 ],
       [0.06593805],
       [0.03782347],
       [0.        ],
       [0.07059312],
       [0.03462717],
       [0.07968191],
       [0.03857584],
       [0.020646  ],
       [0.03197647],
       [0.01893206],
       [0.04279605],
       [0.03979361],
       [0.05059374],
       [0.01539738],
       [0.01800706],
       [0.0184995 ],
       [0.03533326],
       [0.        ],
       [0.01879115],
       [0.03510393],
       [0.12309149],
       [0.        ],
       [0.04543109],
       [0.09829464],
       [0.10193473],
       [0.062177  ],
       [0.02197935],
       [0.11200358],
       [0.05493732],
       [0.04188539],
       [0.03503922],
       [0.06100889],
       [0.09866713],
       [0.07699905],
       [0.02475369],
       [0.03209979],
       [0.05309942],
       [0.11826248],
       [0.        ],
       [0.02207554],
       [0.0402259 ],
       [0.097

Now, we need to find the most similar results. To help with this, we can use the [argsort function from numpy](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html), which will give the indices sorted by value. 

Use the argsort function to find the indices of the 5 most similar articles. Inspect their titles and abstracts. **Warning:** argsort sorts from smallest to largest.

In [48]:

similarity_score_sorted = np.argsort(similarity_scores.flatten())
np.set_printoptions(threshold=np.inf)
similarity_score_sorted[-5:]

array([394, 486,  83, 289, 259], dtype=int64)

In [52]:
articles.loc[similarity_score_sorted[-5:], 'title'].tolist()

['Scan, Materialize, Simulate: A Generalizable Framework for Physically Grounded Robot Planning',
 'Explaining Neural Networks with Reasons',
 'WikiDBGraph: Large-Scale Database Graph of Wikidata for Collaborative Learning',
 'SCENIR: Visual Semantic Clarity through Unsupervised Scene Graph Retrieval',
 'MIRB: Mathematical Information Retrieval Benchmark']

Try using a tfidf vectorizer. How do the results compare?

In [53]:
tfidf = TfidfVectorizer()
X2 = tfidf.fit_transform(articles['abstract'])
y2 = tfidf.transform(query_list)
similarity_scores2 = cosine_similarity(X2, y2)
similarity_score_sorted2 = np.argsort(similarity_scores2.flatten())
np.set_printoptions(threshold=np.inf)
similarity_score_sorted2[-5:]

array([486, 233, 289,  83, 259], dtype=int64)

In [55]:
articles.loc[similarity_score_sorted2[-5:], 'title'].tolist()

['Explaining Neural Networks with Reasons',
 'HDLxGraph: Bridging Large Language Models and HDL Repositories via HDL Graph Databases',
 'SCENIR: Visual Semantic Clarity through Unsupervised Scene Graph Retrieval',
 'WikiDBGraph: Large-Scale Database Graph of Wikidata for Collaborative Learning',
 'MIRB: Mathematical Information Retrieval Benchmark']

### Method 2: Using a Pretrained Embedding Model

Now, let's compare how we do using the [all-MiniLM-L6-v2 embedding model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

This will create a 384-dimensional dense embedding of each sentence.

In [15]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\solid\anaconda3\envs\sentence_transformers\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\solid\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = embedder.encode(sentences)
print(embeddings)

[[ 6.76569417e-02  6.34959564e-02  4.87131029e-02  7.93048963e-02
   3.74480709e-02  2.65277619e-03  3.93749177e-02 -7.09846616e-03
   5.93614057e-02  3.15370224e-02  6.00980818e-02 -5.29052354e-02
   4.06067409e-02 -2.59308442e-02  2.98428535e-02  1.12691487e-03
   7.35148937e-02 -5.03818579e-02 -1.22386597e-01  2.37028375e-02
   2.97265276e-02  4.24768776e-02  2.56337691e-02  1.99514348e-03
  -5.69190495e-02 -2.71597989e-02 -3.29035297e-02  6.60248995e-02
   1.19007193e-01 -4.58791330e-02 -7.26214200e-02 -3.25840116e-02
   5.23414016e-02  4.50553410e-02  8.25305004e-03  3.67024243e-02
  -1.39415674e-02  6.53918684e-02 -2.64272075e-02  2.06393175e-04
  -1.36643527e-02 -3.62811014e-02 -1.95043907e-02 -2.89738420e-02
   3.94270532e-02 -8.84090662e-02  2.62423488e-03  1.36713171e-02
   4.83062454e-02 -3.11566181e-02 -1.17329188e-01 -5.11690676e-02
  -8.85287672e-02 -2.18963325e-02  1.42986281e-02  4.44167778e-02
  -1.34816021e-02  7.43392631e-02  2.66382936e-02 -1.98762678e-02
   1.79191

In [19]:
type(sentences)

list

Use this new embedder to vectorize the abstracts and then find the most similar to the query. How do the results compare to the other methods?

**Warning:** Creating embeddings for all of the articles may take a while.

In [56]:
article_embeddings = embedder.encode(articles['abstract'])
query_embeddings = embedder.encode(query_list)

In [58]:
similarity_scores3 = cosine_similarity(article_embeddings, query_embeddings)
similarity_score_sorted3 = np.argsort(similarity_scores3.flatten())
similarity_score_sorted3[-5:]

array([247, 209, 233, 107, 259], dtype=int64)

In [60]:
articles.loc[similarity_score_sorted3[-5:], 'title'].tolist()

['Distance Adaptive Beam Search for Provably Accurate Graph-Based Nearest Neighbor Search',
 'The Atlas of In-Context Learning: How Attention Heads Shape In-Context Retrieval Augmentation',
 'HDLxGraph: Bridging Large Language Models and HDL Repositories via HDL Graph Databases',
 'Attributing Response to Context: A Jensen-Shannon Divergence Driven Mechanistic Study of Context Attribution in Retrieval-Augmented Generation',
 'MIRB: Mathematical Information Retrieval Benchmark']

### FAISS

The [Faiss library](https://faiss.ai/index.html) is a library for efficient similarity search and clustering of dense vectors. It can be used to automate the process of finding the most similar abstracts.

If we want to use cosine similarity, we need to use the Inner Product. We also need to normalize our vectors so that they all have length 1.

Use the [normalize function](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html) to normalize both the abstract vectors and the query vector.

In [61]:
abstract_norm = normalize(article_embeddings)
query_norm = normalize(query_embeddings)

In [62]:
print(abstract_norm.shape[1])
print(query_norm.shape[1])

384
384


Now, create an [IndexFlatIP object](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes#summary-of-methods) that has dimensions equal to the dimensionality of your vectors. Then add your normalized abstract vectors.

Hint: You can mimic the example [here](https://github.com/facebookresearch/faiss/wiki/Getting-started#building-an-index-and-adding-the-vectors-to-it), but substitute in the IndexFlatIP class.

In [63]:
index = faiss.IndexFlatIP(384)   
print(index.is_trained)
index.add(abstract_norm)                  
print(index.ntotal)

True
500


Finally, use the [search function](https://github.com/facebookresearch/faiss/wiki/Getting-started#searching) on your index object to find the 5 most similar articles.

In [65]:
k = 5  
D, I = index.search(query_norm, k)     
print(I[:5])                  
print(I[-5:])                  


[[259 107 233 209 247]]
[[259 107 233 209 247]]


In [67]:
articles.loc[I[0], 'title'].tolist()

['MIRB: Mathematical Information Retrieval Benchmark',
 'Attributing Response to Context: A Jensen-Shannon Divergence Driven Mechanistic Study of Context Attribution in Retrieval-Augmented Generation',
 'HDLxGraph: Bridging Large Language Models and HDL Repositories via HDL Graph Databases',
 'The Atlas of In-Context Learning: How Attention Heads Shape In-Context Retrieval Augmentation',
 'Distance Adaptive Beam Search for Provably Accurate Graph-Based Nearest Neighbor Search']

In [68]:
index.search(query_norm, k)

(array([[0.47817037, 0.45547354, 0.45146084, 0.43434593, 0.4144665 ]],
       dtype=float32),
 array([[259, 107, 233, 209, 247]], dtype=int64))